In [ ]:
import os
import glob
import socket
import getpass
import yaml
from pathos.multiprocessing import ProcessingPool as Pool
import argparse
from importlib import import_module
from operator import attrgetter

from hyperopt import hp, fmin, tpe, Trials
from hyperopt.mongoexp import MongoTrials

from physopt.objective.utils import PRETRAINING_PHASE_NAME, EXTRACTION_PHASE_NAME, READOUT_PHASE_NAME
from physopt.data_space import build_data_spaces
from physopt.utils.grid_search import suggest
from physopt.config import get_cfg_defaults, get_cfg_debug

from physopt.opt import resolve_config_file, resolve_output_dir, check_cfg,\
get_cfg_from_args, setup_environment_vars, MissingEnvironmentVariable

**setup the environment variables -- for configs and results path**

In [ ]:
NO_PARAM_SPACE = hp.choice('dummy', [0])
CONFIG_ENV_VAR = 'PHYSOPT_CONFIG_DIR'
OUTPUT_ENV_VAR = 'PHYSOPT_OUTPUT_DIR'
setup_environment_vars()

**Load the config file -- using example.yaml here**

In [ ]:
def arg_parse():
    parser = argparse.ArgumentParser(description='Large-scale physics prediction')
    parser.add_argument('-C', '--config', type=str, help='path to physopt configuration file')
    parser.add_argument('-D', '--debug', action='store_true', help='debug mode')
    parser.add_argument('-O', '--output', type=str, help='Output directory for physopt artifacts')
    return parser.parse_args('')

In [ ]:
args = arg_parse()
args.config = './configs/example.yaml'
cfg = get_cfg_from_args(args)

**Setup data loaders**

In [ ]:
data_spaces = build_data_spaces(cfg.DATA_SPACE.MODULE, cfg.DATA_SPACE.FUNC, cfg.DATA_SPACE.SEEDS, cfg.DATA_SPACE.KWARGS)

#set pretraining loader and readout loader
data_space = data_spaces[0]
seed, pretraining_space, readout_space = data_space['seed'], data_space[PRETRAINING_PHASE_NAME], data_space[READOUT_PHASE_NAME][0]

In [ ]:
readout_space

In [ ]:
readout_space

**Pretraining the forward predictor** 

In [ ]:
args = [seed, pretraining_space, None, cfg.CONFIG, cfg.PRETRAINING]
Objective = getattr(import_module(cfg.PRETRAINING.OBJECTIVE_MODULE), cfg.PRETRAINING.OBJECTIVE_NAME)
objective = Objective(*args)
trials = Trials()
#run the optimization
best = fmin(
            objective,
            space=NO_PARAM_SPACE, trials=trials,
            algo=suggest, max_evals=1,
            )

**Extracting latent representations using the pretrained model**

In [ ]:
args = [seed, pretraining_space, readout_space, cfg.CONFIG, cfg.PRETRAINING,  cfg.EXTRACTION]
Objective = getattr(import_module(cfg.EXTRACTION.OBJECTIVE_MODULE), cfg.EXTRACTION.OBJECTIVE_NAME)
objective = Objective(*args)
trials = Trials()
#run the optimization
best = fmin(
            objective,
            space=NO_PARAM_SPACE, trials=trials,
            algo=suggest, max_evals=1,
            )

**Train and test OCP readout model**

In [ ]:
args = [seed, pretraining_space, readout_space, cfg.CONFIG, cfg.PRETRAINING, cfg.EXTRACTION, cfg.READOUT]
Objective = getattr(import_module(cfg.READOUT.OBJECTIVE_MODULE), cfg.READOUT.OBJECTIVE_NAME)
objective = Objective(*args)
trials = Trials()
#run the optimization
best = fmin(
            objective,
            space=NO_PARAM_SPACE, trials=trials,
            algo=suggest, max_evals=1,
            )